In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import os

import neuralNets

def oneHotEncodeData3Classes(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 3))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 1:
            Y_val[j, 1] = 1
        elif targets[j] == 2:
            Y_val[j, 2] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

In [ ]:
tic = time.time()
#training_data = pd.read_csv("appendedTrainingData_test12021-01-31_2021-03-21.csv")
training_data = pd.read_csv("data/numerai_datasets_04.04.21/numerai_training_data.csv")
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

training_data[feature_cols] = training_data[feature_cols].astype(np.float16)
training_data.target        = training_data.target.astype(np.float16)

# X_train = training_data[feature_cols].to_numpy()
# Y_train = training_data.target.to_numpy()

In [ ]:
tournament_data = pd.read_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")  
tournament_data[feature_cols] = tournament_data[feature_cols].astype(np.float16)
tournament_data.target        = tournament_data.target.astype(np.float16)

validation_data = tournament_data.loc[tournament_data.data_type == 'validation']
X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
#X_pred = tournamend_data.loc[tournament_data.data_type == ""]
Y_val = validation_data.target.to_numpy()

X_val_3class = training_data[feature_cols]
Y_val_3class = training_data.target

Y_val_3class = Y_val_3class.replace(1, 0)
Y_val_3class = Y_val_3class.replace([0.25, 0.75], 1)
Y_val_3class = Y_val_3class.replace(0.5, 2)

X_val_3class = X_val_3class.to_numpy()
Y_val_3class = Y_val_3class.to_numpy()


In [ ]:
validation_data = pd.read_csv("data/numerai_datasets_04.04.21/numerai_validation_data.csv")  

X_val_3class = validation_data[feature_cols]
Y_val_3class = validation_data.target

Y_val_3class = Y_val_3class.replace(1, 0)
Y_val_3class = Y_val_3class.replace([0.25, 0.75], 1)
Y_val_3class = Y_val_3class.replace(0.5, 2)

X_val_3class = X_val_3class.to_numpy()
Y_val_3class = Y_val_3class.to_numpy()

In [ ]:
#tournament_data.to_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")

# train classifier between 0/1 0.25/0.75 and 0.5

In [ ]:
X_train_3class = training_data[feature_cols]
Y_train_3class = training_data.target

Y_train_3class = Y_train_3class.replace(1, 0)
Y_train_3class = Y_train_3class.replace([0.25, 0.75], 1)
Y_train_3class = Y_train_3class.replace(0.5, 2)

X_train_3class = X_train_3class.to_numpy()
Y_train_3class = Y_train_3class.to_numpy()

X_train_3class, Y_train_3class = shuffle(X_train_3class, Y_train_3class)

#X_train_3class, X_test_3class, Y_train_3class, Y_test_3class = train_test_split(X_train_3class, Y_train_3class, test_size = 0.1)

In [ ]:
classes = np.array([0, 1, 2])
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1, 2]), Y_train_3class)
w_array = np.ones(Y_train_3class.shape[0], dtype = 'float')
for i, val in enumerate(Y_train_3class):
    index = np.where(classes == val)
    w_array[i] = class_weights[index]
    
print(class_weights)
            
evalset = [(X_train_3class, Y_train_3class), (X_test_3class, Y_test_3class)]
        
earlyStopping = 10
  
retrainModel = True
#model = XGBClassifier(learning_rate=0.3, n_estimators = 1000, max_depth=6, gamma = 5, colsample_bytree=0.5) 
#[999]	validation_0-mlogloss:0.78046	validation_1-mlogloss:0.97374
#model = XGBClassifier(learning_rate=0.3, n_estimators = 1000, max_depth=10, gamma = 5, colsample_bytree=0.5) 
#[393]	validation_0-mlogloss:0.65456	validation_1-mlogloss:0.95505
#model = XGBClassifier(learning_rate=0.1, n_estimators = 1000, max_depth=20, gamma = 5, reg_alpha=0.01, colsample_bytree=0.5)
#[522]	validation_0-mlogloss:0.59919	validation_1-mlogloss:0.93002

#no weights
#model = XGBClassifier(learning_rate=0.1, n_estimators = 1000, max_depth=20, gamma = 5, reg_alpha=0.01, colsample_bytree=0.5)
#[76]	validation_0-mlogloss:0.65330	validation_1-mlogloss:0.89610
model = XGBClassifier(learning_rate=0.1, n_estimators = 1000, max_depth=6, gamma = 1, reg_alpha=0.01, colsample_bytree=0.5)

if retrainModel:
    model.fit(X_train_3class, Y_train_3class,  eval_set=evalset, early_stopping_rounds=earlyStopping) #sample_weight=w_array,
    retrainModel = False
# else:
#     model.fit(X_trainE, Y_trainE,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping, xgb_model='model_eraWise_xgBoost.model')
model.save_model('model_0_1_xgBoost.model')

In [ ]:
modelNN_3classes = neuralNets.defineNN_3classes(X_val_3class.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.89, beta_2=0.99)
modelNN_3classes.compile(optimizer=optAdam, loss='categorical_crossentropy', metrics='categorical_accuracy')
modelNN_3classes.load_weights("model_3class_99train_97val_4_4_21.h5")

In [ ]:
Y_val_3class_oneHot = oneHotEncodeData3Classes(Y_val_3class)

start_date = date(2021,1, 31)
path = "data/numerai_datasets_"
test_date = start_date
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData =6
epochs = 1

epoch = 0

for epoch in range(epochs):
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)
        
        if test_date != date(2021, 4, 4):
            path_to_csv_val = test_path + "/numerai_validation_data.csv" 
            if os.path.exists(path_to_csv_val):
                val_data = pd.read_csv(str(path_to_csv_val))
                val_data_3classes = val_data[training_data.columns]
                
                training_data = pd.concat([training_data, val_data_3classes], axis = 0, ignore_index = True)
        else:
            break


        X_train_3class = training_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
        Y_train_3class = training_data.target        
        
        Y_train_3class = Y_train_3class.replace(1, 0)
        Y_train_3class = Y_train_3class.replace([0.25, 0.75], 1)
        Y_train_3class = Y_train_3class.replace(0.5, 2)        
        
        Y_train_3class_oneHot = oneHotEncodeData3Classes(Y_train_3class)
        #Y_test_3class_oneHot  = oneHotEncodeData3Classes(Y_test_3class)

        class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1, 2]), Y_train_3class)
        class_weights = dict(enumerate(class_weights))
    
        history = modelNN_3classes.fit(X_train_3class, Y_train_3class_oneHot, epochs = epochsPerData, class_weight=class_weights, batch_size = 256*128, validation_data=(X_val_3class, Y_val_3class_oneHot))

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
#import pdb; pdb.set_trace()
#model.save_weights("model.h5")

In [ ]:
modelNN_3classes.save_weights("model_3class_99train_99val_4_4_21.h5")


In [ ]:
pred = modelNN_3classes.predict(X_val_3class)
pred = np.argmax(pred, axis = 1) #convert one hot vectors to labels

accuracy = np.array(Y_val_3class == pred).astype(float).sum() / X_val_3class.shape[0]
print(accuracy)
predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

In [ ]:
pred = modelNN_3classes.predict(X_train_3class)
pred = np.argmax(pred, axis = 1) #convert one hot vectors to labels

accuracy = np.array(Y_train_3class == pred).astype(float).sum() / X_train_3class.shape[0]
print(accuracy)
predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

In [ ]:
predictions_df["target"] = Y_train_3class
predictions_df.target.hist(bins = 10)

# train classifier between 0.25/0.75

In [ ]:
validation_data_025 = validation_data.loc[validation_data.target == 0.25]
validation_data_075 = validation_data.loc[validation_data.target == 0.75]
validation_data_025075 = pd.concat([validation_data_025, validation_data_075], axis = 0, ignore_index = True)


X_val_025075 = validation_data_025075[feature_cols].to_numpy()
Y_val_025075 = validation_data_025075.target
Y_val_025075 = Y_val_025075.replace(0.25, 0)
Y_val_025075 = Y_val_025075.replace(0.75, 1)
Y_val_025075 = Y_val_025075.to_numpy()

X_val_025075, Y_val_025075 = shuffle(X_val_025075, Y_val_025075)

In [ ]:
modelNN_025075 = neuralNets.defineNN_2classes(X_val_025075.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999)
modelNN_025075.compile(optimizer=optAdam, loss='binary_crossentropy', metrics='accuracy')
modelNN_025075.load_weights('model_025075_class_96train__96val.h5')

In [ ]:
start_date = date(2021,1, 31)
path = "data/numerai_datasets_"
test_date = start_date
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData = 10
epochs = 1

epoch = 0

for epoch in range(epochs):
    
    start_date = date(2021,1, 31)
    path = "data/numerai_datasets_"
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)
        
        training_data_075 = training_data.loc[training_data.target == 0.75]
        training_data_025 = training_data.loc[training_data.target == 0.25]
        training_data_025075 = pd.concat([training_data_075, training_data_025], axis = 0, ignore_index = True)

        if test_date != date(2021, 4, 4):
            path_to_csv_val = test_path + "/numerai_validation_data.csv" 
            if os.path.exists(path_to_csv_val):
                val_data = pd.read_csv(str(path_to_csv_val))
                val_data_075 = val_data.loc[val_data.target == 0.75]
                val_data_025 = val_data.loc[val_data.target == 0.25]
                val_data_025075 = pd.concat([val_data_075, val_data_025], axis = 0, ignore_index = True)
                val_data_025075 = val_data_025075[training_data.columns]
                
                training_data_025075 = pd.concat([training_data_025075, val_data_025075], axis = 0, ignore_index = True)

     
        X_train_025075 = val_data_025075[feature_cols].to_numpy()
        Y_train_025075 = val_data_025075.target
        Y_train_025075 = Y_train_025075.replace(0.25, 0)
        Y_train_025075 = Y_train_025075.replace(0.75, 1)
        Y_train_025075 = Y_train_025075.to_numpy()
        
        X_train_025075, Y_train_025075 = shuffle(X_train_025075, Y_train_025075)

        class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_025075)
        class_weights = dict(enumerate(class_weights))


    
        history = modelNN_025075.fit(X_train_025075, Y_train_025075, epochs = epochsPerData, batch_size = 256*128, class_weight=class_weights, validation_data=(X_val_025075, Y_val_025075))

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
        #import pdb; pdb.set_trace()
        #model.save_weights("model.h5")

In [ ]:
modelNN_025075.save_weights("model_025075_class_998train__998val.h5")

In [ ]:
def max_accuracyNP(target, pred):
    correctVals = 0
    n_examples = target.shape[0]
    trueArray = (np.rint(pred) == np.reshape(target, (n_examples,1))).astype(float)
    wrongValues = 0
    for i in range(len(trueArray)):
        if trueArray[i] == 1:
            correctVals += 1
        elif trueArray[i] == 0:
            wrongValues += 1
        else:
            print("weird value ", trueArray[i])
#     print(trueArray.shape, pred.shape, target.shape)
#     for n in range(len(trueArray)):
#         print(pred[n], target[n])
    print("true", correctVals,"wrong", wrongValues, "n_examples", n_examples)
    return trueArray.sum()/n_examples    

def max_accuracy(target, pred):
  """Calculates how often the max prediction matches one-hot labels."""
  retVal = 0
  #print(pred.shape[0])
  if pred.shape[0] != None:  
    num_correct_classified = (tf.math.round(pred) == target)
    num = tf.reduce_sum(tf.dtypes.cast(num_correct_classified, tf.int32), axis = -1)
    retVal = num / pred.shape[0]
    print(tf.math.round(pred))#, num_currect_classified, retVal)
  else:
    retVal = tf.dtypes.cast(tf.math.round(pred) == target, tf.int32)
  print(retVal)
  return retVal

In [ ]:
pred = modelNN_025075.predict(X_val_025075)

print(max_accuracyNP(Y_val_025075, pred))

predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

# train classifier between 0 and 1

In [ ]:
training_data_1 = training_data.loc[training_data.target == 1]
training_data_0 = training_data.loc[training_data.target == 0]
training_data_01 = pd.concat([training_data_1, training_data_0], axis = 0, ignore_index = True)

X_train_01 = training_data_01[feature_cols].to_numpy()
Y_train_01 = training_data_01.target.to_numpy()
X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)

X_train_01, X_test_01, Y_train_01, Y_test_01 = train_test_split(X_train_01, Y_train_01, test_size = 0.1)

In [ ]:
validation_data_1 = validation_data.loc[validation_data.target == 1]
validation_data_0 = validation_data.loc[validation_data.target == 0]
validation_data_01 = pd.concat([validation_data_1, validation_data_0], axis = 0, ignore_index = True)

X_val_01 = validation_data_01[feature_cols].to_numpy()
Y_val_01 = validation_data_01.target.to_numpy()
#X_val_01, Y_val_01 = shuffle(X_val_01, Y_val_01)

In [ ]:
classes = np.array([0, 1])
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
w_array = np.ones(Y_train_01.shape[0], dtype = 'float')
for i, val in enumerate(Y_train_01):
    index = np.where(classes == val)
    w_array[i] = class_weights[index]
    
print(class_weights)
            
evalset = [(X_train_01, Y_train_01), (X_val_01, Y_val_01)]
        
earlyStopping = 10
  
retrainModel = True
model = XGBClassifier(learning_rate=0.005, n_estimators = 1000, max_depth=10, gamma = 25)#, colsample_bytree=0.1)    

if retrainModel:
    model.fit(X_train_01, Y_train_01,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping)
    retrainModel = False
# else:
#     model.fit(X_trainE, Y_trainE,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping, xgb_model='model_eraWise_xgBoost.model')
model.save_model('model_0_1_xgBoost.model')

In [ ]:
def defineNN_2classes(n_inputFeatures):
    activation = "relu"
    regularizationConst_l1 = 0.00000#3
    regularizationConst_l2 = 0.00000#3
    size = 512
    X_input = Input(shape=(n_inputFeatures,))
    X = Dropout(0.00, input_shape = (n_inputFeatures,))(X_input)
 
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    #X = BatchNormalization(axis = -1)(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    #X = BatchNormalization(axis = -1)(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)

    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    #X = BatchNormalization(axis = -1)(X)
    X = Dense(size/4, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/8, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/16, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/32, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)

    X = Dense(1, activation="sigmoid")(X)
    
    model = Model(inputs = X_input, outputs = X, name='deepNN')

    return model


modelNN_01 = defineNN_2classes(X_train_01.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
modelNN_01.compile(optimizer=optAdam, loss='binary_crossentropy', metrics='accuracy')
#modelNN_01.load_weights('model_01class_97train_97test.h5')

In [ ]:
start_date_orig = date(2021,1, 31)
#start_date_orig = date(2021,3, 28)
path = "data/numerai_datasets_"
test_date = start_date_orig
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

epochsPerData = 25
epochs = 1
counter = 0
epoch = 0

for epoch in range(epochs):
    
    start_date = start_date_orig
    path = "data/numerai_datasets_"
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)

        
        training_data_1 = training_data.loc[training_data.target == 1]
        training_data_0 = training_data.loc[training_data.target == 0]
        training_data_01 = pd.concat([training_data_1, training_data_0], axis = 0, ignore_index = True)
        
        if test_date != date(2021, 4, 4):
            path_to_csv_val = test_path + "/numerai_validation_data.csv" 
            if os.path.exists(path_to_csv_val):
                val_data = pd.read_csv(str(path_to_csv_val))
                val_data_1 = val_data.loc[val_data.target == 1]
                val_data_0 = val_data.loc[val_data.target == 0]
                val_data_01 = pd.concat([val_data_1, val_data_0], axis = 0, ignore_index = True)
                val_data_01 = val_data_01[training_data.columns]
                training_data_01 = pd.concat([training_data_01, val_data_01], axis = 0, ignore_index = True)
        
        #append all training data into one dataframe
        if counter == 0:
            appendedTraining_data_01 = training_data_01
        else:
            appendedTraining_data_01 = pd.concat([training_data_01, appendedTraining_data_01], axis = 0, ignore_index = True)
        
#         X_train_01 = training_data_01[feature_cols].to_numpy()
#         Y_train_01 = training_data_01.target.to_numpy()
#         X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)
        
#         class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
#         class_weights = dict(enumerate(class_weights))


    
#        history = modelNN_01.fit(X_train_01, Y_train_01, epochs = epochsPerData, batch_size = 256*128*2, class_weight=class_weights, validation_data=(X_test_01, Y_test_01))

        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
        counter += 1
        #import pdb; pdb.set_trace()
        #model.save_weights("model.h5")
 

# train with one appended matrix
print(appendedTraining_data_01.shape)
X_train_01 = appendedTraining_data_01[feature_cols].to_numpy()
Y_train_01 = appendedTraining_data_01.target.to_numpy()
X_train_01, Y_train_01 = shuffle(X_train_01, Y_train_01)
        
class_weights = class_weight.compute_class_weight('balanced', np.array([0, 1]), Y_train_01)
class_weights = dict(enumerate(class_weights))


In [ ]:
history = modelNN_01.fit(X_train_01, Y_train_01, epochs = 5, batch_size = 256*128*2, class_weight=class_weights, validation_data=(X_val_01, Y_val_01))


In [ ]:
appendedTraining_data_01.to_csv("appendedTrainingData_classes_0_1_till_04_04_21.csv")

In [ ]:
modelNN_01.save_weights("model_01class_99train_99test_994val.h5")

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
#print(history.history)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
pred = modelNN_01.predict(X_val_01)

print(max_accuracyNP(Y_val_01, pred))

predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)

In [ ]:
test_true = np.array([0, 1, 0])
test_pred = np.array([0.4, 0.500000001, 0.1])


max_accuracy(test_true, test_pred)

In [ ]:
pred = modelNN_01.predict(X_train_01)
#pred2 = modelNN.predict(X_test_01)

print(max_accuracyNP(Y_train_01, pred))

predictions_df = pd.DataFrame(pred)
#predictions_df["pred"] = pred
predictions_df.hist(bins = 10)